## Toy MHD and divergence cleaning 


We made a code to check the divergence cleaning strategies discussed in: https://www.overleaf.com/read/phmtkqtkxwjr#baf932

In this version we add the hyperbolic divergence cleaning. Since we are using periodic boundary conditions we don't expect much improvement, only the one coming from the damping term. 

In [ ]:
using Plots
using SummationByPartsOperators
using LinearAlgebra
using ArraysOfArrays
using Base.Threads
using Revise
using JLD2
#import Pkg; Pkg.add("WriteVTK")
using WriteVTK

includet("../PIC/PIC-1D/aux_functions/aux_functions_grid.jl")
includet("../PIC/PIC-1D/aux_functions/aux_functions_E-B.jl")
includet("../PIC/PIC-1D/aux_functions/aux_functions_RHS.jl")
includet("toy_MHD_auxfunctions.jl")

In [ ]:
J = (100,100)
Box = (0.0,1.0,0.0,1.0)
dx = differentials(Box,J)
const D = 2

xv = [(i-1)*dx[1] for i in 1:J[1]]
yv = [(i-1)*dx[2] for i in 1:J[2]];

In [ ]:
Dx = periodic_derivative_operator(derivative_order=1, accuracy_order=4, xmin=Box[1], xmax=Box[2], N=J[1])
Dy = periodic_derivative_operator(derivative_order=1, accuracy_order=4, xmin=Box[3], xmax=Box[4], N=J[2])
Δx = dissipation_operator(Dx;
                     #mode=D.coefficients.mode
                     #,mode=ThreadedMode()
                     )
Δy = dissipation_operator(Dy;
                     #mode=D.coefficients.mode
                     #,mode=ThreadedMode()
                     )
σx = 0.0 #dissipation strength
σy = 0.0 #dissipation strength

Build initial data and velocity field.

In [ ]:
k1 = 2π*[1,1]
k2 = 2π*[2,2]
x0 = [0.5,0.5]
fv(x,y,k) = sin(k[1]*x)*sin(k[2]*y)
```
bump of radius r and smoothness p around the point x0
```
function fp(x,y,x0,r,p) 
    r20 = r^2
    r2 = (x-x0[1])^2+(y-x0[2])^2
    if r2 - r20 < 0
        return (r20 - r2)^p/r20^p
    else
        return 0.0
    end
end
function grad_fp(x,y,x0,r,p)
    r20 = r^2
    r2 = (x-x0[1])^2+(y-x0[2])^2
    if r2 - r20 < 0
        return -2*p*[(x-x0[1]),(y-x0[2])]*(r20 - r2)^(p-1)/r20^p
    else
        return [0.0, 0.0]
    end
end

#possible V's 

#V = [-(l-2)*sin(k1[1]*xv[i]) + (l-1)*sin(k1[2]*yv[j]) for l in 1:2, i in 1:J[1], j in 1:J[2]];
grad_Fp = [grad_fp(xv[i],yv[j],x0,0.45,8)[l] for l in 1:D, i in 1:J[1], j in 1:J[2]];
V = -0.01*grad_Fp

#possible B's

B = [(l-2)*k2[2]*sin(k2[1]*xv[i])*cos(k2[2]*yv[j]) + (l-1)*k2[1]*cos(k2[1]*xv[i])*sin(k2[2]*yv[j]) for l in 1:2, i in 1:J[1], j in 1:J[2]];


Plot velocity field

In [ ]:
surface(yv,xv,V[1,:,:])

Plot initial magnetic field

In [ ]:
surface(yv,xv,B[2,:,:], title="components of the initial magnetic field")

Check the maximum initial constraint violation.

In [ ]:
@show maximum(abs.(Div(B,Dx,Dy,J)))

Plot the initial constraint violation. Notice that it is of high frequency.

In [ ]:
surface(yv,xv,Div(B,Dx,Dy,J), title="initial divergence of B")

Plot the divergence of the velocity field, some times it is important to see where the divergence of the magnetic field will grow.

In [ ]:
surface(yv,xv,Div(V,Dx,Dy,J), title="divergence of V")

Create the union of evolutionary fields. And location for its time derivatives.

In [ ]:
u = Vector{Float64}(undef,(D+1)*J[1]*J[2]); #the last component is psi
du = Vector{Float64}(undef,(D+1)*J[1]*J[2]);

u .= 0.0;
du .= 0.0;

U = reshape(u,D+1,J[1],J[2]);



#F = B


The evolution equations are:

$$
\partial_t B^i = \nabla_j (v^j B^i - v^i B^j) + \kappa \nabla^i (\nabla_j B^j)
$$
for the parabolic case, and

\begin{align*}
\partial_t B^i &= \nabla_j (v^j B^i - v^i B^j) + \nabla^i ψ \\
\partial_t ψ &= c^2 \nabla_j B^j - τψ
\end{align*}


### Set the method:

In [ ]:
#=
hyperbolic = false
parabolic = false
no_div_cleaning = false
hyperbolic = true
#parabolic = true
no_div_cleaning = true
=#
#method = :hyperbolic
method = :parabolic
#method = :no_div_cleaning
@show method

### Set the initial data.

In [ ]:
U[1:D,:,:] .= copy(B)
U[D+1,:,:] .= 0.0 #psi
@show B === reshape(u,(D+1,J...))[1:D,:,:]
@show norm(B - reshape(u,(D+1,J...))[1:D,:,:])
@show maximum(abs.(u))
@show get_Energy(u,J,Box)
@show maximum(abs.(Div(reshape(u,(D+1,J...))[1:D,:,:],Dx,Dy,J)))

@show typeof(u)
if method == :hyperbolic || method == :no_div_cleaning
@show typeof(F_hyp!(u,0.0,(J, Dx, Δx, Dy, Δy, σx, σy, V, 0.1, 1.0, 1.0, du)))
surface(yv,xv,reshape(F_hyp!(u,0.0,(J, Dx, Δx, Dy, Δy, σx, σy, zeros(D,J...), 0.1, 1, 1, du)),D+1,J...)[3,:,:], title="vector flow")
elseif method == :parabolic
@show typeof(F_par!(u,0.0,(J, Dx, Δx, Dy, Δy, σx, σy, V, 0.1, 1.0, 1.0, du)))
surface(yv,xv,reshape(F_hyp!(u,0.0,(J, Dx, Δx, Dy, Δy, σx, σy, zeros(D,J...), 0.1, 1, 1, du)),D+1,J...)[3,:,:], title="vector flow")
end
#surface(yv,xv,du[2,:,:])

### Set the evolution parameters:

In [ ]:
(t_i, t_f) = (0.0, 2.0)
M = 1600
@show dt = (t_f - t_i)/M
@show differentials(Box,J)

if method == :hyperbolic 
      κ = 0.0
      c2 = 1.0
      τ = 1.0
elseif method == :parabolic
      κ = maximum(dx)
      c2 = 0.0
      τ = 0.0
end

if method == :no_div_cleaning
      κ = 0.0
      c2 = 0.0
      τ = 0.0
end



@show κ, c2, τ
par = (J, Dx, Δx, Dy, Δy, σx, σy, V, κ, c2, τ, du)
t = t_i
Energy = Array{Float64,1}(undef,M+1)
Div_B_max = Array{Float64,1}(undef,M+1)
Div_B_L2 = Array{Float64,1}(undef,M+1)
tt = Array{Float64,1}(undef,M+1)
@show Energy[1] = get_Energy(u,J,Box)
@show Div_B_max[1] = maximum(abs.(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J)))
@show Div_B_L2[1] = norm(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J))*sqrt(volume(Box)/prod(J))
tt[1] = t;

In [ ]:
@show κ, c2, τ

In [ ]:
for i in 1:M
    if method == :hyperbolic || method == :no_div_cleaning
        RK4_Step!(F_hyp!, u, 0.0, dt, par)
    elseif method == :parabolic
        RK4_Step!(F_par!, u, 0.0, dt, par)
    end
    t = t + dt
    tt[i+1] = t
    
    Energy[i+1] = get_Energy(u,J,Box)
    Div_B_max[i+1] = maximum(abs.(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J)))
    Div_B_L2[i+1] = norm(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J))*sqrt(volume(Box)/prod(J))

    println("t = $t")
    println("Energy = $(get_Energy(u,J,Box))")
    println("DivB = $(maximum(abs.(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J))))")
    
end

In [ ]:
surface(yv,xv,Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J), title="final divergence of B")

In [ ]:
#It works in Paraview but not in VisIt
vtk_grid("u", xv, yv) do vtk
    vtk["B"] = @views reshape(u,(D+1,J...))[1:D, :, :]        # Magnetic field attached to points
    vtk["ψ"] = @views reshape(u,(D+1,J...))[3, :, :]     # scalar field attached to cells
    vtk["Velocity"] = V            # vector field attached to points
    vtk["TimeValue"] = tt[M+1]                           # metadata ("field data" in VTK)
end

The ψ field:

In [ ]:
surface(yv,xv,reshape(u,(D+1,J...))[3, :, :], title="final ψ")

The magnetic field:

In [ ]:
surface(yv,xv,reshape(u,(D+1,J...))[1, :, :], title="Final B")

In [ ]:
plot(tt, Energy, yscale=:log10, title="Energy vs time", xlabel="t", ylabel="Energy")

In [ ]:
plot(tt, Div_B_max,title="Max |div B| vs time",xlabel="t",ylabel="Max |div B|")

In [ ]:
plot(tt, Div_B_L2,title="Norm |div B| vs time",xlabel="t",ylabel="Max |div B|")

### Comparison:

We compare both cleaning methods against themselves and the free, no-cleaning evolution.

In [ ]:

if method == :no_div_cleaning 
    Div_B_max_0 = copy(Div_B_max); #for no div control.
    Div_B_L2_0 = copy(Div_B_L2);
else
    if method == :hyperbolic
        Div_B_max_1_1 = copy(Div_B_max);
        Div_B_L2_1_1 = copy(Div_B_L2);
    elseif method == :parabolic
        Div_B_max_1 = copy(Div_B_max);
        Div_B_L2_1 = copy(Div_B_L2);
    end
end


We observe that in $L^2$ norm both the non-cleaning and the hyperbolic methods grow. The hyperbolic does a good job in holding the violation small. But the parabolic methods works much better, keeping the violation to a better level than what it was originally. 

**Warning:** in these simulations the magnetic field evolves very high frequencies very quickly and already the last part of the computation does not resolves the solution very well. Even though the constraint violation stays small.

In [ ]:
plot(tt,Div_B_L2_0, title="Norm |div B| vs time D-order=4",xlabel="t",ylabel="Norm |div B|", label="κ=0.0", yscale=:log10)
plot!(tt,Div_B_L2_1_1, label="c2=1 τ=1")
plot!(tt,Div_B_L2_1, label="κ=1 τ=0")
#savefig("DivB_L2_vs_time_D-order=4_bump_neg.png")

![DivB_L2_vs_time](DivB_L2_vs_time_D-order=4_bump_neg.png)

In the maximum norm we observe something similar. 

In [ ]:
plot(tt,Div_B_max_0, title="Max |div B| vs time D-order=4",xlabel="t",ylabel="Max |div B|", label="c2=0.0", yscale=:log10)
plot!(tt,Div_B_max_1_1, label="c2=1 τ=1")
plot!(tt,Div_B_max_1, label="κ=1 τ=0")
#savefig("DivB_max_vs_time_D-order_bump_neg=4.png")

![DivB_max_vs_time](DivB_max_vs_time_D-order_bump_neg=4.png)